# User 0 

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[User['Nº cartão EVIO'] == 0]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

print(User)

              Start date           Stop date  Duration (min)  \
31   2024-02-25 10:20:00 2024-02-25 10:37:00          16.500   
35   2024-02-23 08:37:00 2024-02-23 11:31:00         174.000   
52   2024-02-20 14:12:00 2024-02-20 20:23:00         370.850   
61   2024-02-19 08:31:00 2024-02-19 11:39:00         188.083   
91   2024-02-11 10:48:00 2024-02-11 11:14:00          25.983   
...                  ...                 ...             ...   
1536 2023-04-03 08:39:00 2023-04-03 13:38:00         298.967   
1542 2023-03-31 08:48:00 2023-03-31 13:36:00         288.500   
1545 2023-03-31 08:41:00 2023-03-31 10:26:00         105.350   
1548 2023-03-31 08:37:00 2023-03-31 08:45:00           8.283   
1559 2023-03-29 08:35:00 2023-03-29 10:26:00         111.117   

      Total Energy (kWh) Nº cartão EVIO    Weekday  
31                 28.52              0     Sunday  
35                 31.10              0     Friday  
52                 27.23              0    Tuesday  
61                 

In [3]:
merged_df = User

print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)


# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')


print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)

print(merged_df)


# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
              Start date           Stop date  Duration (min)  \
31   2024-02-25 10:20:00 2024-02-25 10:37:00          16.500   
35   2024-02-23 08:37:00 2024-02-23 11:31:00         174.000   
52   2024-02-20 14:12:00 2024-02-20 20:23:00         370.850   
61   2024-02-19 08:31:00 2024-02-19 11:39:00         188.083   
91   2024-02-11 10:48:00 2024-02-11 11:14:00          25.983   
...                  ...                 ...             ...   
1536 2023-04-03 08:39:00 2023-04-03 13:38:00         298.967   
1542 2023-03-31 08:48:00 2023-03-31 13:36:00         288.500   
1545 2023-03-31 08:41:00 2023-03-31 10:26:00         105.350   
1548 2023-03-31 08:37:00 2023-03-31 08:45:00           8.283   
1559 2023-03-29 08:35:00 2023-03-29 10:26:00         111.117   

      Total Energy (kWh) Nº cartão EVIO  Weekday  
31                 28.52              0        7  
35                 31.10              0        5  
52                

In [9]:
# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
merged_df.sort_values(by='Start date', inplace=True)

# Calcular a diferença entre o 'Start date' da linha atual e o 'Stop date' da linha anterior em horas
merged_df['Hours_since_last_charge'] = (merged_df['Start date'] - merged_df['Stop date'].shift(1)).dt.total_seconds() / 3600
merged_df['Hours_since_last_charge'].fillna(0, inplace=True)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

              Start date           Stop date  Duration (min)  \
1559 2023-03-29 08:35:00 2023-03-29 10:26:00         111.117   
1548 2023-03-31 08:37:00 2023-03-31 08:45:00           8.283   
1545 2023-03-31 08:41:00 2023-03-31 10:26:00         105.350   
1542 2023-03-31 08:48:00 2023-03-31 13:36:00         288.500   
1536 2023-04-03 08:39:00 2023-04-03 13:38:00         298.967   
...                  ...                 ...             ...   
91   2024-02-11 10:48:00 2024-02-11 11:14:00          25.983   
61   2024-02-19 08:31:00 2024-02-19 11:39:00         188.083   
52   2024-02-20 14:12:00 2024-02-20 20:23:00         370.850   
35   2024-02-23 08:37:00 2024-02-23 11:31:00         174.000   
31   2024-02-25 10:20:00 2024-02-25 10:37:00          16.500   

      Total Energy (kWh) Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
1559               12.60              0        3            1            1   
1548                1.23              0        5            1            0 

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
#hours = range(8, 21)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.7107692307692308
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
1       Carregar      Carregar      Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar      Carregar      Carregar  Não Carregar  Não Carregar   
5   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
6   Não Carregar  Não Carregar      Carregar  Não Carregar  Não Carregar   
7   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
8       Carregar      Carregar      Carregar  Não Carregar  Não Carregar   
9   Não Carregar      Carregar      Carregar  Não Carregar  Não Carregar   
10  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
11  Não Carregar  Não Carregar  Não Ca

In [ ]:
import numpy as np

# Suponha que y_train seja a matriz de treinamento do seu modelo original
# y_train: linhas são instâncias e colunas são horas (de 8h às 20h)

# Inicializar contadores de transição
transitions = np.zeros((2, 2))  # 2 estados: 0 - Não Carregar, 1 - Carregar

# Contar transições entre estados para cada hora
for i in range(len(y_train) - 1):  # Exclui a última linha para evitar index out of bounds
    for j in range(y_train.shape[1] - 1):  # Transições entre horas consecutivas
        current_state = y_train[i, j]
        next_state = y_train[i, j + 1]
        transitions[current_state, next_state] += 1

# Normalizar para obter probabilidades de transição
transition_matrix = transitions / transitions.sum(axis=1, keepdims=True)

print("Matriz de Transição:")
print(transition_matrix)

# Simulação de previsões baseada em Corrente de Markov
def markov_prediction(initial_state, hours, transition_matrix):
    state = initial_state
    predictions = []
    
    for hour in range(hours):
        predictions.append(state)
        state = np.random.choice([0, 1], p=transition_matrix[state])
    
    return predictions

# Exemplo de simulação para 12 horas
initial_state = 0  # Começa não carregando
simulated_predictions = markov_prediction(initial_state, 12, transition_matrix)

print("Previsões Simuladas:", simulated_predictions)


# User 29 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[User['Nº cartão EVIO'] == 29]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

print(User)

              Start date           Stop date  Duration (min)  \
13   2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   
66   2024-02-16 14:15:00 2024-02-16 15:28:00          72.617   
67   2024-02-16 13:48:00 2024-02-16 14:14:00          25.583   
111  2024-02-06 08:34:00 2024-02-06 13:06:00         272.633   
127  2024-02-01 12:24:00 2024-02-01 12:44:00          20.433   
...                  ...                 ...             ...   
1708 2023-02-15 13:53:00 2023-02-15 19:29:00         336.067   
1724 2023-02-14 08:38:00 2023-02-14 12:48:00         250.700   
1730 2023-02-13 08:31:00 2023-02-13 12:59:00         268.833   
1762 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
1771 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   

      Total Energy (kWh) Nº cartão EVIO    Weekday  
13                52.820             29  Wednesday  
66                12.860             29     Friday  
67                33.580             29     Friday  
111               4

In [2]:
merged_df = User

print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)


# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')


print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)

print(merged_df)


# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
              Start date           Stop date  Duration (min)  \
13   2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   
66   2024-02-16 14:15:00 2024-02-16 15:28:00          72.617   
67   2024-02-16 13:48:00 2024-02-16 14:14:00          25.583   
111  2024-02-06 08:34:00 2024-02-06 13:06:00         272.633   
127  2024-02-01 12:24:00 2024-02-01 12:44:00          20.433   
...                  ...                 ...             ...   
1708 2023-02-15 13:53:00 2023-02-15 19:29:00         336.067   
1724 2023-02-14 08:38:00 2023-02-14 12:48:00         250.700   
1730 2023-02-13 08:31:00 2023-02-13 12:59:00         268.833   
1762 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
1771 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   

      Total Energy (kWh) Nº cartão EVIO  Weekday  
13                52.820             29        3  
66                12.860             29        5  
67                

In [3]:
# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
merged_df.sort_values(by='Start date', inplace=True)

# Calcular a diferença entre o 'Start date' da linha atual e o 'Stop date' da linha anterior em horas
merged_df['Hours_since_last_charge'] = (merged_df['Start date'] - merged_df['Stop date'].shift(1)).dt.total_seconds() / 3600
merged_df['Hours_since_last_charge'].fillna(0, inplace=True)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

              Start date           Stop date  Duration (min)  \
1771 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   
1762 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
1730 2023-02-13 08:31:00 2023-02-13 12:59:00         268.833   
1724 2023-02-14 08:38:00 2023-02-14 12:48:00         250.700   
1708 2023-02-15 13:53:00 2023-02-15 19:29:00         336.067   
...                  ...                 ...             ...   
127  2024-02-01 12:24:00 2024-02-01 12:44:00          20.433   
111  2024-02-06 08:34:00 2024-02-06 13:06:00         272.633   
67   2024-02-16 13:48:00 2024-02-16 14:14:00          25.583   
66   2024-02-16 14:15:00 2024-02-16 15:28:00          72.617   
13   2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   

      Total Energy (kWh) Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
1771              41.440             29        3            1            1   
1762              23.820             29        5            1            1 

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
#hours = range(8, 21)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.7032967032967034
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar      Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar      Carregar  Não Carregar  Não Carregar   
5   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
6   Não Carregar  Não Carregar      Carregar  Não Carregar  Não Carregar   
7   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
8   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
9   Não Carregar  Não Carregar      Carregar  Não Carregar  Não Carregar   
10  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
11  Não Carregar  Não Carregar  Não Ca

In [5]:
import numpy as np

# Suponha que y_train seja a matriz de treinamento do seu modelo original
# y_train: linhas são instâncias e colunas são horas (de 8h às 20h)

# Inicializar contadores de transição
transitions = np.zeros((2, 2))  # 2 estados: 0 - Não Carregar, 1 - Carregar

# Contar transições entre estados para cada hora
for i in range(len(y_train) - 1):  # Exclui a última linha para evitar index out of bounds
    for j in range(y_train.shape[1] - 1):  # Transições entre horas consecutivas
        current_state = y_train[i, j]
        next_state = y_train[i, j + 1]
        transitions[current_state, next_state] += 1

# Normalizar para obter probabilidades de transição
transition_matrix = transitions / transitions.sum(axis=1, keepdims=True)

print("Matriz de Transição:")
print(transition_matrix)

# Simulação de previsões baseada em Corrente de Markov
def markov_prediction(initial_state, hours, transition_matrix):
    state = initial_state
    predictions = []
    
    for hour in range(hours):
        predictions.append(state)
        state = np.random.choice([0, 1], p=transition_matrix[state])
    
    return predictions

# Exemplo de simulação para 12 horas
initial_state = 0  # Começa não carregando
simulated_predictions = markov_prediction(initial_state, 12, transition_matrix)

print("Previsões Simuladas:", simulated_predictions)


Matriz de Transição:
[[0.92653673 0.07346327]
 [0.25622776 0.74377224]]
Previsões Simuladas: [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]


In [9]:
import numpy as np

# Supondo que a matriz de transição já tenha sido calculada (substitua pela sua matriz)
transition_matrix = np.array([[0.92653673, 0.07346327], [0.25622776, 0.74377224]])

# Função para gerar previsões usando a matriz de transição
def markov_prediction(initial_state, transition_matrix, hours):
    state = initial_state
    predictions = []
    
    for _ in range(hours):
        predictions.append(state)
        state = np.random.choice([0, 1], p=transition_matrix[state])
    
    return predictions

# Gerar previsões para todo o conjunto de teste
markov_predictions = []

for i in range(len(X_test)):  # Para cada instância no conjunto de teste
    # Suponha que o estado inicial seja baseado em y_test ou outra heurística
    initial_state = y_test[i, 0]  # Começa no estado da primeira hora do dia
    predictions = markov_prediction(initial_state, transition_matrix, len(hours))  # 12 horas de previsão
    markov_predictions.append(predictions)

# Converter para array para facilitar a comparação
markov_predictions = np.array(markov_predictions)

# Calcular a acurácia do modelo de Markov
markov_accuracy = accuracy_score(y_test.ravel(), markov_predictions.ravel())
print("Acurácia do modelo de Markov:", markov_accuracy)

# Matriz de confusão para o modelo de Markov
markov_conf_matrix = confusion_matrix(y_test.ravel(), markov_predictions.ravel())
print("Matriz de Confusão - Markov:\n", markov_conf_matrix)

# Relatório de classificação para o modelo de Markov
print("Relatório de Classificação - Markov:\n", classification_report(y_test.ravel(), markov_predictions.ravel()))


Acurácia do modelo de Markov: 0.663003663003663
Matriz de Confusão - Markov:
 [[151  41]
 [ 51  30]]
Relatório de Classificação - Markov:
               precision    recall  f1-score   support

           0       0.75      0.79      0.77       192
           1       0.42      0.37      0.39        81

    accuracy                           0.66       273
   macro avg       0.59      0.58      0.58       273
weighted avg       0.65      0.66      0.66       273



In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Previsões do modelo de regressão logística já calculadas
logistic_predictions = predictions  # Suas previsões originais

# Previsões do modelo de Markov (simulação)
# Suponha que você tenha armazenado as previsões de Markov em uma variável `markov_predictions`
# markov_predictions deve ter o mesmo formato que logistic_predictions

# Acurácia do modelo de regressão logística
logistic_accuracy = accuracy_score(y_test.ravel(), logistic_predictions.ravel())
print("Acurácia do modelo de Regressão Logística:", logistic_accuracy)

# Acurácia do modelo de Markov
markov_accuracy = accuracy_score(y_test.ravel(), markov_predictions.ravel())
print("Acurácia do modelo de Markov:", markov_accuracy)

# Matriz de confusão para o modelo de regressão logística
logistic_conf_matrix = confusion_matrix(y_test.ravel(), logistic_predictions.ravel())
print("Matriz de Confusão - Regressão Logística:\n", logistic_conf_matrix)

# Matriz de confusão para o modelo de Markov
markov_conf_matrix = confusion_matrix(y_test.ravel(), markov_predictions.ravel())
print("Matriz de Confusão - Markov:\n", markov_conf_matrix)

# Relatório de classificação para o modelo de regressão logística
print("Relatório de Classificação - Regressão Logística:\n", classification_report(y_test.ravel(), logistic_predictions.ravel()))

# Relatório de classificação para o modelo de Markov
print("Relatório de Classificação - Markov:\n", classification_report(y_test.ravel(), markov_predictions.ravel()))


Acurácia do modelo de Regressão Logística: 0.7032967032967034


NameError: name 'markov_predictions' is not defined